<a href="https://colab.research.google.com/github/Yuxin-Zhang-Jasmine/DeepUS_ABLE/blob/development/version2_1_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import scipy.io as scio
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
tf.__version__ 

'2.4.1'

## 1-3. Prepare x_train / y_train / y_train_weights

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip '/content/drive/MyDrive/DeepUS/datasets.zip'

Archive:  /content/drive/MyDrive/DeepUS/datasets.zip
  inflating: datasets.hdf5           


In [6]:
f = h5py.File('datasets.hdf5','r')
pht = list(f['PICMUS16']['x_train'].keys())
# print(pht)  ['carotid_cross_expe',..., 'resolution_distorsion_simu']

In [22]:
x_train = np.array(f['PICMUS16']['x_train'][pht[4]])  # np.amax(x_train) == 0.1356
y_train = np.array(f['PICMUS16']['y_train'][pht[4]])  # np.amax(y_train) == 105.612274
y_train_weights = np.array(f['PICMUS16']['y_train_weights'][pht[4]])

In [ ]:
# x_train = np.vstack((f['PICMUS16']['x_train'][pht[i]] for i in range(2)))
# y_train = np.vstack((f['PICMUS16']['y_train'][pht[i]] for i in range(2)))

# 1. Reproduce ABLE

In [8]:
Ne = 128 # number of transducer elements

## 1-1. Build a model

In [9]:
def antirectifier(x):   #copy from https://www.tensorflow.org/.../keras/layers/Lambda
    x -= K.mean(x, axis=1, keepdims=True)
    x = K.l2_normalize(x, axis=1)
    pos = K.relu(x)
    neg = K.relu(-x)
    return K.concatenate([pos, neg], axis=1)

In [10]:
_input = keras.Input(shape=(Ne,))
x = keras.layers.Dense(Ne)(_input)
x = keras.layers.Lambda(antirectifier)(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(Ne/4)(x)
x = keras.layers.Lambda(antirectifier)(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(Ne/4)(x)
x = keras.layers.Lambda(antirectifier)(x)
x = keras.layers.Dropout(0.2)(x)

weights = keras.layers.Dense(Ne, name = "weights")(x)  # no activation is applied
sumWeights = keras.layers.Lambda(lambda x: K.sum(x, axis=1, keepdims=True), name = "sumWeights")(weights) 

multiplied = keras.layers.Multiply()([_input, weights])
output = keras.layers.Lambda(lambda x: K.sum(x, axis=1, keepdims=True), name="output")(multiplied)

model = keras.Model(
    inputs=[_input],
    outputs=[output,sumWeights],
)

#uncomment to see the architecture of the model
#keras.utils.plot_model(model, "ABLE_model_with_shape_info.png", show_shapes=True)  

## 1-2 Define loss functions and compile

In [11]:
def loss_SMSLE(y_true, y_pred, scale=1.):
    first_log = K.log(K.clip(K.abs(y_pred[:, :1]) * scale, K.epsilon(), None) + 1.) * K.sign(y_pred[:, :1])
    second_log = K.log(K.clip(K.abs(y_true[:, :1]) * scale, K.epsilon(), None) + 1.) * K.sign(y_true[:, :1])
    return K.mean(K.square(first_log - second_log), axis=-1)

In [12]:
def loss_unity(y_true, y_pred):
    return K.mean( K.square(y_pred - y_true),  axis=0 )

In [13]:
lbd = 0.7
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=1e-3),
    loss={"output":loss_SMSLE, "sumWeights":loss_unity},
    loss_weights=[lbd,1-lbd]
)

## 1-3. Train and save the model

In [14]:
val_split = 0.3
EPOCHS = 10
checkpoint_filepath = '/content/model_saved'

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

early_stop_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=2)

In [27]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [26]:
# Model weights are saved at the end of every epoch, if it's the best seen so far.
model.fit(x=x_train, y=[y_train, y_train_weights], 
          validation_split=val_split, epochs=EPOCHS, batch_size=235683,
          callbacks=[model_checkpoint_callback,early_stop_callback])

# The training result here corresponds to batch_size=32

Epoch 1/10
386668/386668 [==============================] - 2191s 6ms/step - loss: 0.4767 - output_loss: 0.6796 - sumWeights_loss: 0.0031 - val_loss: 0.2064 - val_output_loss: 0.2948 - val_sumWeights_loss: 8.7867e-06
Epoch 2/10
386668/386668 [==============================] - 2171s 6ms/step - loss: 0.1985 - output_loss: 0.2823 - sumWeights_loss: 0.0030 - val_loss: 0.1805 - val_output_loss: 0.2502 - val_sumWeights_loss: 0.0177
Epoch 3/10
386668/386668 [==============================] - 2174s 6ms/step - loss: 0.1761 - output_loss: 0.2503 - sumWeights_loss: 0.0031 - val_loss: 0.1578 - val_output_loss: 0.2252 - val_sumWeights_loss: 3.5345e-04
Epoch 4/10
386668/386668 [==============================] - 2173s 6ms/step - loss: 0.1614 - output_loss: 0.2293 - sumWeights_loss: 0.0030 - val_loss: 0.1477 - val_output_loss: 0.2109 - val_sumWeights_loss: 2.8497e-04
Epoch 5/10
386668/386668 [==============================] - 2177s 6ms/step - loss: 0.1526 - output_loss: 0.2167 - sumWeights_loss: 0.003

In [28]:
model.save("my_model.tf")
model.save_weights("my_weights.tf")  #use it beacause checkpoint doesn't work

INFO:tensorflow:Assets written to: my_model.tf/assets
